In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using ForwardDiff
using Test
using RobotZoo
import RobotDynamics
using LinearAlgebra
using StaticArrays
using SparseArrays
using Printf
using MeshCat
using Plots
using FiniteDiff
import MathOptInterface as MOI
using Ipopt
using JLD2

include("quadratic_cost.jl")   # defines the quadratic cost function type
include("q2_model.jl")         # sets up the dynamics
const isautograder = @isdefined autograder

  Activating environment at `~/Classes/16745_OptimalControl/hw3_solutions/Project.toml`


false

# Q1: Sequential Quadratic Programming (SQP) (50 pts)
In this problem you'll solve the canonical cartpole swing-up problem using the classic direct collocation algorithm with Hermite-Simpson integration. 

### Continuous Problem
We will be solving a trajectory optimization problem of the form:
$$
\begin{aligned} 
&\underset{x(t), u(t)}{\text{minimize}} && J_f(x(t_f)) + \int_{t_0}^{t_f} J(x(t), u(t)) dt \\
&\text{subject to} && \dot{x}(t) = f(x(t), u(t), t) \\
&&& x(t_0) = x_\text{init} \\
&&& x(t_f) = x_\text{goal}
\end{aligned} 
$$

### Hermite-Simpson Collocation
Recall from lecture that direct collocation "transcribes" the continuous-time optimal control problem into a finite-dimensional nonlinear program (NLP). We will use Hermite-Simpson integration on both our dynamics and our cost function. We will split our 
cost integral into $N-1$ segments of length $h$ seconds, and approximate the cost 
over this interval using a Hermite spline:

$$ \int_{t_k}^{t_{k+1}} J\big(x(t),u(t)\big) dt \approx 
\frac{h}{6}\bigg(
J\big(x(t_k), u(t_k)\big) + 
4 J\big(x(t_k + h/2), u(t_k + h/2)\big) + 
J\big(x(t_k + h), u(t_k + h)\big) \bigg) $$

where we calculate the state at the midpoint with:
$$ x(t_k + h/2) = x_m = \frac{1}{2} \big(x_1 + x_2 \big) + 
\frac{h}{8}\big(f(x_k, u_k, t_k) - f(x_{k+1}, u_{k+1}, t_{k+1}) \big) $$

and we use first-order-hold on the controls:
$$ u(t_k + h/2) = u_m = \frac{1}{2} \big( u_1 + u_2 \big) $$

For our dynamics constraint, we use implicit integration with the same Hermite spline:
$$ \frac{h}{6} \big(f(x_k,u_k,t_k) + 4f(x_m,u_m,t_m) + f(x_{k+1}, u_{k+1}, t_{k+1}) \big) + x_k - x_{k+1} = 0 $$

### Discrete Problem
The resulting NLP has the following form:
$$
\begin{aligned} 
&\underset{x_{1:N}, u_{1:N}}{\text{minimize}} && J_f(x_N) + 
\sum_{k=1}^{N-1} \frac{h}{6}(J(x_k,u_k) + 4J(x_m,u_m) + J(x_{k+1}, u_{k+1}))  \\
&\text{subject to} && \frac{h}{6} \big(f(x_k,u_k,t_k) + 4f(x_m,u_m,t_m) + f(x_{k+1}, u_{k+1}, t_{k+1}) \big) + x_k - x_{k+1} = 0 \\
&&& x_1 = x_\text{init} \\
&&& x_N = x_\text{goal}
\end{aligned} 
$$

Note that the state midpoint is really a function of the states and controls at the surrounding knot points: $x_m(x_k, u_k, x_{k+1}, u_{k+1}, t_k, h)$ and the control at the midpoint is a function of the previous and next and control values: $u_m(u_k, u_{k+1})$. You will need differentiate through these splines using the chain rule to generate the methods we need to solve our NLP.

### Solving the Problem
To make things easier, we'll use Ipopt to solve our NLP, but you'll still need to define the functions we pass to Ipopt. Ipopt expects a problem of the following form:

$$
\begin{aligned} 
&\underset{x}{\text{minimize}} && f(x) \\
&\text{subject to} && l \leq c(x) \leq u\\
\end{aligned} 
$$

Since our problem only has equality constraints, our upper and lower bounds $u$ and $l$ will both be zero. Ipopt requires that we specify analytical functions that evaluate $\nabla f$ and $\nabla c$. For best performance, the function evaluating the constraint Jacobian typically only evaluates the nonzero elements. To make things simple, we treat the Jacobian as dense. 

This homework problem will give you valuable experience in setting up the optimization problems in a way that can be passed to off-the-shelf NLP solvers like Ipopt.

## The Problem
You likely have already seen the cartpole swing-up problem previously.The system is comprised of a pendulum attached to a cart, where forces can only be applied to the cart. The goal is to balance the pendulum above the cart. The system dynamics can be written as:

$$ x = \begin{bmatrix} y \\ \theta \\ v \\ \omega \end{bmatrix}, \quad \dot{x} = \begin{bmatrix} \dot{q} \\ \ddot{q} \end{bmatrix}, \quad
q = \begin{bmatrix} y \\ \theta \end{bmatrix}, \quad
\ddot{q} = -H^{-1} (C \dot{q} + G - B u)$$

where 
$$ H = \begin{bmatrix} m_c + m_p & m_p l \cos{\theta} \\
m_p l \cos{\theta} & m_p l^2 \end{bmatrix}, \;
C = \begin{bmatrix} 0 & -m_p \omega l \sin{\theta} \\ 0 & 0 \end{bmatrix}, \;
G = \begin{bmatrix} 0 \\ m_p g l \sin{\theta} \end{bmatrix}, \;
B = \begin{bmatrix} 1 \\ 0 \end{bmatrix} $$

with the following parameters:
* $m_p$: mass of the pole
* $m_c$: mass of the cart
* $g$: gravity
* $l$: length of the rod

Our goal is to move the cart in a way that we get the pendulum to swing from the downward position (`[0,0,0,0]`) to an upright position (`[0,pi,0,0]`).

We've encapsulated all of the problem information into a `struct` for convenience (and to avoid polluting our global workspace with uncessary global variables). 


## Developing in External Editor
All of the methods you need to implement in this problem are in external Julia files.
Feel free to use a text editor / IDE of your choice (the Julia VSCode extension is the IDE recommended by the Julia community) to write and test these methods. You can use the `q2.jl` script to run the code, which includes tests that are identical to those in this notebook. We will be running the notebooks for for the autograder, so before you submit make sure this notebook runs as expected and passes the tests (or run `test/runtests.jl` which will run the autograder).

In [2]:
include("q2_prob.jl")  # Defines a struct containing all of the problem information

get_initial_trajectory

In [3]:
prob = CartpoleProblem();

In [4]:
let model = prob.model
    isautograder && return
    global vis = Visualizer()
    set_mesh!(vis, model)
    render(vis)
end

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/brian/.julia/packages/MeshCat/Ax8pH/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("rod" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x72, 0x6f, 0x64], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "cart" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("pole" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x63, 0x61, 0x72, 0x74, 0x2f, 0x70, 0x6f, 0x6c, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("cyl" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x65, 0x2f, 0x67, 0x65, 0x6f, 0x6d, 0x2f, 0x63, 0x79, 0x6c], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "mass" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x67, 0x65, 0x6f, 0x6d, 0x2f, 0x6d, 0x61, 0x73, 0x73], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))), "box" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x63, 0x61, 0x72, 0x74, 0x2f, 0x62, 0x6f, 0x78], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "hinge" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x63, 0x61, 0x72, 0x74, 0x2f, 0x68, 0x69, 0x6e, 0x67, 0x65], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))))), Set{Any}(), ip"127.0.0.1", 8700))

In [5]:
let X = get_initial_trajectory(prob)[1]
    isautograder || visualize!(vis, prob.model, prob.tf, X)
end

## Part (a): Write Cost Functions (15 pts)
Our first task will be to write methods to evaluate our objective / cost function. We first create a `struct` that will be responsible for evaluating all the functions we need to pass to Ipopt.

In [21]:
include("sparseblocks.jl")  # SOLUTION
include("q2_nlp.jl")

unpackZ

In [22]:
?NLP

search: NLP nlp nullspace TriangleP LineLoop sequential_palette



```
NLP{n,m,L,Q}
```

Represents a (N)on(L)inear (P)rogram of a trajectory optimization problem, with a dynamics model of type `L`, a quadratic cost function, horizon `T`,  and initial and final state `x0`, `xf`.

The kth state and control can be extracted from the concatenated state vector `Z` using `Z[nlp.xinds[k]]`, and `Z[nlp.uinds[k]]`.

# Constructor

```
NLP(model, obj, tf, T, x0, xf, [integration])
```

# Basic Methods

```
Base.size(nlp)    # returns (n,m,T)
num_ineq(nlp)     # number of inequality constraints
num_eq(nlp)       # number of equality constraints
num_primals(nlp)  # number of primal variables
num_duals(nlp)    # total number of dual variables
packZ(nlp, X, U)  # Stacks state `X` and controls `U` into one vector `Z`
```

# Evaluating the NLP

The NLP supports the following API for evaluating various pieces of the NLP:

```
eval_f(nlp, Z)         # evaluate the objective
grad_f!(nlp, grad, Z)  # gradient of the objective
hess_f!(nlp, hess, Z)  # Hessian of the objective
eval_c!(nlp, c, Z)     # evaluate the constraints
jac_c!(nlp, c, Z)      # constraint Jacobian
```


### Useful Examples
You may find the following code snippets helpful as you complete the methods for the NLP.

In [23]:
let
    # Create NLP
    nlp = NLP(prob)

    # Create a vector of all states and controls
    X,U = get_initial_trajectory(prob)
    Z = packZ(nlp, X, U)

    # Unpack into states and vectors
    X2, U2 = unpackZ(nlp, Z)

    # Get kth state, control
    k = 10
    x = Z[nlp.xinds[k]]
    u = Z[nlp.uinds[k]]

    # Dynamics
    t = nlp.times[k]
    dt = nlp.times[k+1] - nlp.times[k]
    dynamics(nlp.model, x, u, t)

    # Dynamics Jacobian
    A,B = dynamics_jacobians(nlp.model, x, u, t);
end;

## Objective
**TASK** Finish the following methods defined in this file included in the cell below:
- `eval_f` (5 pts)
- `grad_f!` (10 pts)

The docstrings for these function are printed below. You will be graded on the number of function and Jacobian evaluations you use. You should avoid unnecessary dynamics and dynamics Jacobian evaluations. You should only need a maximum `N + (N-1)` evaluations each of the dynamics and dynamics Jacobians for each function.

**TIP**: You may find it helpful to define some helper function that evaluate all of the terms you need upfront. We've provided some example starter code in `q2_dynamics.jl`. Feel free to include that file and modify as needed. You can also add fields to the `NLP` struct if you feel the need (the TA's solution uses the provided fields).

In [24]:
# SOLUTION
include("q2_dynamics.jl")

include("q2_cost_methods.jl")

hess_f!

In [25]:
?eval_f

search: eval_f eval_c! evaluate_dynamics! evaluate_midpoints!



```
eval_f(nlp, Z)
```

Evaluate the objective, returning a scalar. The continuous time objective is of the form:

$$
\int_{t0}^{tf} \ell(x(t), u(t)) dt 
$$

You need to approximate this with an integral of the form:

$$
\sum_{k=1}^{N-1} \frac{h}{6}(\ell(x_k,u_k) + 4\ell(x_m, u_m) + \ell(x_{k+1}, u_{k+1}))
$$

where

$$
x_m = \frac{1}{2} (x_1 + x_2) + \frac{h}{8}(f(x_1, u_1, t) - f(x_2, u_2, t + h))
$$

and

$$
u_m = \frac{1}{2} (u_1 + u_2)
$$


In [26]:
?grad_f!

search: grad_f!



```
grad_f!(nlp, grad, Z)
```

Evaluate the gradient of the objective at `Z`, storing the result in `grad`.


In [27]:
@testset "Q2a" begin                                               # POINTS = 15
    prob = CartpoleProblem()
    nlp = NLP(prob)
    X,U = get_initial_trajectory(prob) 
    Z = packZ(nlp, X, U)

    @testset "eval_f" begin                                        # POINTS = 5
        # Test the cost
        @test eval_f(nlp, Z) ≈ 0.22766546346850902 atol=1e-6       # POINTS = 3
        devals = @dynamicsevals eval_f(nlp, Z)
        @test devals <= 301                                        # POINTS = 0.5
        @test devals <= 201                                        # POINTS = 1
        @test devals >= 200                                        # POINTS = 0.5
    end

    @testset "grad_f" begin                                        # POINTS = 10
        # Test the cost gradient with FiniteDiff
        grad = zero(Z)
        grad_f!(nlp, grad, Z)
        devals = @dynamicsevals grad_f!(nlp, grad, Z)
        jevals = @jacobianevals grad_f!(nlp, grad, Z)
        @test devals <= 301                                        # POINTS = 0.5
        @test devals <= 201                                        # POINTS = 1
        @test devals >= 200                                        # POINTS = 0.5
        @test jevals <= 301                                        # POINTS = 0.5
        @test jevals <= 201                                        # POINTS = 1
        @test jevals >= 200                                        # POINTS = 0.5
        
        grad_fd = FiniteDiff.finite_difference_gradient(x->eval_f(nlp, x), Z)
        @test norm(grad - grad_fd) < 1e-8                          # POINTS = 6
    end    
end;

Test Summary: | Pass  Total
Q2a           |   11     11


# Part (b): Evaluate the Constraints (20 pts)
Next, we need to define functions to evaluate our constraints. We should have `n + (N-1) n + n` constraints, since we have an initial and goal state, and `(N-1)` dynamics constraints of size `n`, where `n` is the size of our state vector (4).
The vector should be stacked as follows:

$$ \begin{bmatrix}
x_1 - x_\text{init} \\
\frac{h}{6}(f(x_1, u_1, t_1) + 4 f(x_m, u_m, t_m) + f(x_2, u_2, t_2) + x_1 - x_2 \\
\vdots \\
\frac{h}{6}(f(x_{N-1}, u_{N-1}, t_{N-1}) + 4 f(x_m, u_m, t_m) + f(x_N, u_N, t_N) + x_{N-1} - x_N \\
x_N - x_\text{goal}
\end{bmatrix} $$

**TASK**: Complete the following functions defined in the file included in the cell below:
- eval_c!(nlp, c, Z)
- jac_c!(nlp, jac, Z)

As with the cost functions, you will be graded on how many dynamics function evaluations you use. You should only need $N + (N-1)$ dynamics evaluations for the 
constraints and $N + (N-1)$ dynamics Jacobian evaluations for the constraint Jacobian.

You are **NOT** allowed to use finite differencing or automatic differentiation in this function. Not only should you be familiar with how to apply the chain rule to get the pieces you need analytically, we already use ForwardDiff to get the dynamics Jacobians, and nesting calls to ForwardDiff usually results in poor performance.

**TIPS**: 
- Don't worry about the number of dynamics / Jacobian evaluations to begin with. Do something that works, then worry about "performance."
- Consider writing some helper functions to evaluate all the pieces you need before the loop. These will probably be the same as the ones you needed for the cost functions.
- Write out the derivatives you need by hand using the chain rule. Cache the individual pieces of the chain rule you need and then multiply them together to get the final 
Jacobians.
- Check intermediate Jacobians (e.g. the Jacobians for a single dynamics constraint) with ForwardDiff or FiniteDiff to make sure you've applied the chain rule correctly, then apply it in a loop.

The docstrings for these functions are printed below.

In [28]:
include("q2_constraints.jl")

initialize_sparsity! (generic function with 1 method)

In [29]:
?eval_c!

search: eval_c! evaluate_dynamics! evaluate_dynamics_jacobians!



```
eval_c!(nlp, c, Z)
```

Evaluate the equality constraints at `Z`, storing the result in `c`. The constraints should be ordered as follows: 

1. Initial condition $x_1 = x_\text{init}$
2. Hermite Simpson Dynamics: $\frac{h}{6} (f(x_k, u_k) + 4 f(x_m, u_m) + f(x_{k+1}, u_{k+1})) + x_k - x_{k+1} = 0$
3. Terminal constraint $x_N = x_\text{goal}$

Consider leveraging the caches in `nlp` to evaluate the dynamics and the midpoints  before the main loop, so that you can making redundant calls to the dynamics.

Remember, you will loose points if you make more dynamics calls than necessary.  Start with something that works, then think about how to eliminate any redundant  dynamics calls.


In [30]:
?jac_c!

search: jac_c! CARTPOLE_JACOBIAN_CACHE



```
jac_c!(nlp, jac, Z)
```

Evaluate the constraint Jacobian, storing the result in the matrix `jac`. You will need to apply the chain rule to calculate the derivative of the dynamics constraints with respect to the states and controls at the current and next time  steps.

### Use of automated differentiation tools

You are not allowed to use automatic differentiation methods for this function.  You are only allowed to call `dynamics_jacobians` (which under the hood does use ForwardDiff). You are allowed to check your answer with these tools, but your final  solution should not use them.


In [31]:
@testset "Q2b" begin                                              # POINTS = 20
    prob = CartpoleProblem()
    nlp = NLP(prob)
    X,U = get_initial_trajectory(prob) 
    Z = packZ(nlp, X, U)

    resfile = joinpath(@__DIR__, "Q2.jld2")

    @testset "eval_c" begin                                      # POINTS = 8
        # Constraint function
        c = zeros(num_duals(nlp))
        devals = @dynamicsevals eval_c!(nlp, c, Z)
        @test devals <= 301                                      # POINTS = 0.5
        @test devals <= 201                                      # POINTS = 1
        @test devals >= 200                                      # POINTS = 0.5
        @test norm(c - load(resfile, "c0")) < 1e-8               # POINTS = 6
    end
    

    @testset "jac_c" begin                                       # POINTS = 12 
        # Calc constraint Jacobian and check Jacobian evals
        jac = zeros(num_duals(nlp), num_primals(nlp))
        jevals = @jacobianevals jac_c!(nlp, jac, Z)
        devals = @dynamicsevals jac_c!(nlp, jac, Z)
        @test devals <= 301                                      # POINTS = 0.5
        @test devals <= 201                                      # POINTS = 1
        @test devals <= 101                                      # POINTS = 0.5
        @test devals == 0                                        # POINTS = 1
        @test jevals <= 301                                      # POINTS = 0.5
        @test jevals <= 201                                      # POINTS = 1
        @test jevals >= 200                                      # POINTS = 0.5

        # Check constraint Jacobian with FiniteDiff
        jac_fd = zero(jac)
        FiniteDiff.finite_difference_jacobian!(jac_fd, (y,x)->eval_c!(nlp, y, x), Z)
        @test norm(jac - jac_fd) < 1e-6                          # POINTS = 7
    end
end;

Test Summary: | Pass  Total
Q2b           |   12     12


## Part (c): Solving the NLP (5 pts)
Now that we have the methods we need to evaluate our NLP, we can solve it with Ipopt. 
We use [`MathOptInterface.jl`](https://github.com/jump-dev/MathOptInterface.jl) to interface with the Ipopt solver. Don't worry too much about this interface: we take care of all of the boilerplate code in the file below.

You don't need to do anything for this part: if you all of your methods above are correct, your problem should converge in about 30 iterations. If your problem isn't converging, go check your methods above. Remember, the tests aren't perfect and won't catch all of your mistakes. Debugging these types of solvers is a critical skill that takes practice.

In [32]:
include("q2_moi.jl")

solve

In [33]:
prob = CartpoleProblem()
X,U = get_initial_trajectory(prob)
nlp = NLP(prob)
Z0 = packZ(nlp, X, U)
Zsol,solver = solve(Z0, nlp)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:   206040
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      505
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      408
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du 

([0.0, 0.0, 0.0, 0.0, 4.329722269439634, 0.0009033667164502093, -0.0018055760882920871, 0.09220173497048365, -0.18416959773666922, 4.892793111778058  …  -0.0007613789690813062, 3.1400688744391934, 0.07419692934839114, 0.14859560859502483, -3.416201121383158, 0.0, 3.141592653589793, -6.175459658415095e-31, 0.0, -4.001476086965557], Ipopt.Optimizer)

In [34]:
isautograder || render(vis)

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("rod" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x72, 0x6f, 0x64], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "cart" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("pole" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x63, 0x61, 0x72, 0x74, 0x2f, 0x70, 0x6f, 0x6c, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("cyl" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x65, 0x2f, 0x67, 0x65, 0x6f, 0x6d, 0x2f, 0x63, 0x79, 0x6c], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "mass" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x67, 0x65, 0x6f, 0x6d, 0x2f, 0x6d, 0x61, 0x73, 0x73], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))), "box" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x63, 0x61, 0x72, 0x74, 0x2f, 0x62, 0x6f, 0x78], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "hinge" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x63, 0x61, 0x72, 0x74, 0x2f, 0x68, 0x69, 0x6e, 0x67, 0x65], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))))), Set(Any[WebSocket(server, CONNECTED), WebSocket(server, CONNECTED)]), ip"127.0.0.1", 8700))

In [35]:
let X = [Zsol[xi] for xi in nlp.xinds]
    isautograder || visualize!(vis, prob.model, prob.tf, X)
end

In [36]:
@testset "Q2c" begin                                     # POINTS = 5
    Z = copy(Zsol)
    λ = MOI.get(solver, MOI.NLPBlockDual()) # get the duals
    X,U = unpackZ(nlp, Zsol)
    @test norm(X[1] - prob.x0) < 1e-6                    # POINTS = 0.5
    @test norm(X[end] - prob.xf) < 1e-6                  # POINTS = 0.5
    grad = zeros(num_primals(nlp))
    grad_f!(nlp, grad, Z)
    c = zeros(num_duals(nlp))
    eval_c!(nlp, c, Z)
    jac = spzeros(num_duals(nlp), num_primals(nlp))
    jac_c!(nlp, jac, Z)
    @test norm(grad - jac'λ, Inf) < 1e-6                 # POINTS = 2
    @test norm(c, Inf) < 1e-6                            # POINTS = 2
end;

Test Summary: | Pass  Total
Q2c           |    4      4


## Part (d): Track the solution with model error (10 pts)
Let's now use our trajectory and simulate it on a system with some model mismatch.

**TASK**:
1. Generate controller that tracks your optimized trajectories.
2. Run your controller on a simulated cartpole with a cart mass of 1.5 kg instead of 1 kg. Get it to successfully stabilize. The final stabilized position doesn't have to to be at an x-position of 0. Simulate for at least 10 seconds.

**TIPS**:
1. Feel free to use code from previous homeworks. 
2. It will stabilize with TVLQR
3. If your cartpole gets it to the top but doesn't stabilize it for the full 10 seconds, think about how you could design your controller to stabilize it about the unstable equilibrium...

In [37]:
include("q2_controller.jl")

gen_controller

In [38]:
?gen_controller

search: gen_controller



```
gen_controller(nlp, Zref)
```

Create a controller that tracks the output of the NLP solver, `Zref`. The `ctrl` object you output should support a function with the following signature:

```
get_control(ctrl, x, t)
```

that returns the control `u` given the state vector `x` and time `t`.

You are free to implement any controller that satisfies this signature  (LQR, TVLQR, MPC, a learned policy, etc.) The only requirements are that it achieves the  swing-up for a cartpole mass of 1.5 kg (the reference was designed with a mass of 1.0 kg), and runs in faster than real time.

Before you try anything crazy, consider the simplest controller that can probably achieve  this goal.


In [39]:
isautograder || render(vis)

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("rod" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x72, 0x6f, 0x64], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "cart" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("pole" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x63, 0x61, 0x72, 0x74, 0x2f, 0x70, 0x6f, 0x6c, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("cyl" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x65, 0x2f, 0x67, 0x65, 0x6f, 0x6d, 0x2f, 0x63, 0x79, 0x6c], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "mass" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x67, 0x65, 0x6f, 0x6d, 0x2f, 0x6d, 0x61, 0x73, 0x73], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))), "box" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x74, 0x2f, 0x63, 0x61, 0x72, 0x74, 0x2f, 0x62, 0x6f, 0x78], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "hinge" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x63, 0x61, 0x72, 0x74, 0x2f, 0x68, 0x69, 0x6e, 0x67, 0x65], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))))), Set(Any[WebSocket(server, CONNECTED), WebSocket(server, CONNECTED), WebSocket(server, CONNECTED)]), ip"127.0.0.1", 8700))

In [40]:
# Simulate with a different model
let Zref = copy(Zsol)
    ctrl = gen_controller(nlp, Zref)
    model2 = RobotZoo.Cartpole(1.1, 0.2, 0.5, 9.81)
    Xsim, Usim, tsim = simulate(model2, nlp.x0, ctrl, tf=5nlp.tf, dt=0.005)
    isautograder || visualize!(vis, model2, tsim[end], Xsim)
end

Converged in 384 iterations
Controller ran at 19205.89839342612 Hz


In [41]:
using Random
@testset "Q2d" begin                                                # POINTS = 10
    Random.seed!(1)
    model2 = RobotZoo.Cartpole(1.1, 0.2, 0.5, 9.81)
    ctrl = gen_controller(nlp, Zsol)
    tsim = @elapsed Xsim, Usim, tsim = 
        simulate(model2, nlp.x0, ctrl, tf=5nlp.tf, dt=0.005)
    
    # Test real-time performance
    @test tsim < 5nlp.tf
    
    # Check that it gets to the goal
    @test abs(Xsim[end][1]) < 0.1
    @test abs(Xsim[end][2] - pi) < 1e-2
    @test abs(Xsim[end][3]) < 0.1
    @test abs(Xsim[end][4]) < 1e-2 
    @test norm(Usim[end-10:end], Inf) < 0.3
end;

Converged in 384 iterations
Controller ran at 2.289429152979862e6 Hz
Test Summary: | Pass  Total
Q2d           |    6      6


# Part (e): EXTRA CREDIT Leveraging sparsity (max 5 pts)
NLP solvers like Ipopt or SNOPT are designed to leverage sparsity in the problem, especially in the constraint Jacobian. Right now we're ignoring the sparsity structure in the constraint Jacobian, so the solver iterations are fairly slow. Get up to 5 extra credit points by leveraging the sparsity structure in the constraint Jacobian. You'll need to read up on how to specify the sparsity pattern in the [MathOptInterface.jl documentation](http://jump.dev/MathOptInterface.jl/stable/reference/nonlinear/). You should only leverage the sparsity when the `use_sparse_jacobian` flag in the `NLP` struct is set to true. We use this flag to compare the solutions between the normal (dense) version and your sparse version. You'll get points for having matching Jacobians, the size of the nonzeros vector you're passing to Ipopt (shoot for a sparsity of less than 2-5%). You'll also get up to 2 points for the speedup you get from the solver (the TA solution got a speed up of about 100x).

**TIPS**
- You'll need to modify the `MOI.jacobian_structure` method in `q2_moi.jl`
- You'll need to modify the `jac_c!` method that takes a vector in `q2_constraints.jl`

We will run the following function to calculate your extra credit:

In [42]:
include("q2_tests.jl");

In [43]:
isautograder && println("Running extra credit")
extra_credit_points = test_extracredit()

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:   206040
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      505
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      408
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du 

5